In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.environ['OPENAI_API_KEY']

In [16]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo")


In [22]:
from langchain_community.utilities import SQLDatabase
sqlite_db_path = "data/street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [23]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)
chain.invoke({"question": "How many trees are in the database?"})

'SELECT COUNT(id) AS tree_count\nFROM trees;'

In [24]:
db.run(response)

NameError: name 'response' is not defined

In [ ]:
chain.get_prompts()[0].pretty_print

<bound method StringPromptTemplate.pretty_print of PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use

In [ ]:
from langchain_community.tools.sql_database import QuerySQLDataBaseTool
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)
chain = write_query | execute_query
chain.invoke({"question": "How many trees are in the database?"}) 

## Translate sql response into natural language response

In [27]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

answer_prompt = PromptTemplate.from_template(
    """Given the following user question,
    corresponding SQL query, and SQL result,
    answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

write_query = create_sql_query_chain(llm, db)

execute_query = QuerySQLDataBaseTool(db=db)

#chain = write_query | execute_query

#chain.invoke({"question": "List the species of trees that are present in San Francisco"})

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

/var/folders/90/s9rwqz1j6ng3m5ghghjb6rp80000gp/T/ipykernel_44209/1726508684.py:27: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


'The SQL query retrieves the species of trees that are present in San Francisco up to a limit of 5 rows. To provide the full list of tree species in San Francisco, you can remove the LIMIT clause from the query and execute it again.'